In [ ]:
### Without Pre Process

In [1]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
#from keras.applications.vgg19 import preprocess_input                
#from keras.applications.resnet50 import preprocess_input            
#from keras.applications.inception_resnet_v2 import preprocess_input               
from keras.applications.mobilenet import preprocess_input               

from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

#from keras.applications.xception import Xception

#from keras.applications.inception_v3 import InceptionV3

#from keras.applications.vgg19 import VGG19

#from keras.applications.resnet50 import ResNet50   

#from keras.applications.inception_resnet_v2 import InceptionResNetV2    

from keras.applications.mobilenet import MobileNet    

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_MobileNet_data4.h5'  
train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception_ResNet_V2 network

    model = applications.MobileNet(weights='imagenet', include_top=False)

    model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_MobileNet_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_MobileNet_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_MobileNet_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_MobileNet_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_validation shape',y_pred_validation.shape)
    print(' y_pred_validation : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/keras/applications/mobilenet.py:224: UserWarning: MobileNet shape is undefined. Weights for input shape 

17227776/17225924 [==============================] - 12s 1us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (Activation)      (None, None, None, 32)    0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 

Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

Epoch 39/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.6488 - acc: 0.8290 - val_loss: 2.5588 - val_acc: 0.8325
Epoch 40/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.8704 - acc: 0.8140 - val_loss: 2.8829 - val_acc: 0.8175
Epoch 41/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.9177 - acc: 0.8110 - val_loss: 3.4210 - val_acc: 0.7800
Epoch 42/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.7974 - acc: 0.8200 - val_loss: 2.7332 - val_acc: 0.8225
Epoch 43/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.7206 - acc: 0.8260 - val_loss: 3.0110 - val_acc: 0.8075
Epoch 44/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.7384 - acc: 0.8270 - val_loss: 2.6157 - val_acc: 0.8350
Epoch 45/50
1000/1000 [==============================] - 16s 16ms/step - loss: 2.7387 - acc: 0.8260 - val_loss: 2.9022 - val_acc: 0.8175
Epoch 46/50
1000/1000 [==================

 y_pred_validation shape (400,)
 y_pred_validation : 
  [0.0000000e+00 1.0160267e-30 1.0000000e+00 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.2900493e-23 5.9943617e-26 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 6.9096137e-17 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 9.9999142e-01 0.0000000e+00 0.0000000e+00 8.1873943e-31 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000

In [ ]:
### With Pre Process

In [2]:
import tensorflow as tf 
import keras
import os

from subprocess import check_output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
#from keras.applications.vgg16 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
#from keras.applications.vgg19 import preprocess_input                
#from keras.applications.resnet50 import preprocess_input            
#from keras.applications.inception_resnet_v2 import preprocess_input               
from keras.applications.mobilenet import preprocess_input               

from keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

from sklearn.cross_validation import train_test_split

from keras.utils import np_utils

#from keras.applications.vgg16 import VGG16

#from keras.applications.xception import Xception

#from keras.applications.inception_v3 import InceptionV3

#from keras.applications.vgg19 import VGG19

#from keras.applications.resnet50 import ResNet50   

#from keras.applications.inception_resnet_v2 import InceptionResNetV2    

from keras.applications.mobilenet import MobileNet    

from keras.models import Model, load_model

from keras.layers import Dense, Dropout, Reshape

from keras.optimizers import SGD

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report,confusion_matrix

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten 
from keras import backend as K
from keras import applications


os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)


# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model_MobileNet_PP_data4.h5'  
train_data_dir = '../data4_Train500_Test200/train'
validation_data_dir = '../data4_Train500_Test200/validation'
nb_train_samples = 1000
nb_validation_samples = 400
epochs = 50
batch_size = 8  # batch size in flow_images_from_directory needs to correspond to the image number of the test images.


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception_ResNet_V2 network

    model = applications.MobileNet(weights='imagenet', include_top=False)

    model.summary()
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DR1_bottleneck_features_train_MobileNet_PP_data4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DR1_bottleneck_features_validation_MobileNet_PP_data4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DR1_bottleneck_features_train_MobileNet_PP_data4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    preprocess_input(train_data)   
    
    print(' train_labels shape',train_labels.shape)

    print(' train_labels : \n ',train_labels)

    validation_data = np.load(open('DR1_bottleneck_features_validation_MobileNet_PP_data4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    preprocess_input(validation_data)   

    print(' validation_labels shape',validation_labels.shape)

    print(' validation_labels : \n ',validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    print('-------------------------------------------------------------------')
    print('------------------ Evalute the train set ------------------------')
    print('-------------------------------------------------------------------')

    train_score = model.evaluate(train_data, train_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('Train loss:', train_score[0])
    print('Train accuracy:', train_score[1])

    y_pred_train = np.squeeze(model.predict(train_data))
    print(' y_pred_train shape',y_pred_train.shape)
    print(' y_pred_train : \n ',y_pred_train)

    threshold = 0.5
    y_pred_train = (y_pred_train > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_train again: \n ',y_pred_train)
    
    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=True)
    print('accuracy_score_train with normalize=True: ', accuracy_score_train)

    accuracy_score_train = accuracy_score(train_labels, y_pred_train, normalize=False)
    print('accuracy_score_train with normalize=False : ', accuracy_score_train)

    target_names = ['class 0', 'class 1']

    print(classification_report(y_pred_train, train_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_train, train_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )

    print('-------------------------------------------------------------------')
    print('------------------ Evalute the validation set ------------------------')
    print('-------------------------------------------------------------------')

    validation_score = model.evaluate(validation_data, validation_labels, verbose=0) # nb_train_samples // batch_size, workers=1)
    print('validation loss:', validation_score[0])
    print('validation accuracy:', validation_score[1])

    y_pred_validation = np.squeeze(model.predict(validation_data))
    print(' y_pred_validation shape',y_pred_validation.shape)
    print(' y_pred_validation : \n ',y_pred_validation)

    y_pred_validation = (y_pred_validation > threshold)*1
    #y_pred_train.astype(int)   
    print(' y_pred_validation again: \n ',y_pred_validation)
    
    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=True)
    print('accuracy_score_validation with normalize=True: ', accuracy_score_validation)

    accuracy_score_validation = accuracy_score(validation_labels, y_pred_validation, normalize=False)
    print('accuracy_score_validation with normalize=False : ', accuracy_score_validation)

    print(classification_report(y_pred_validation, validation_labels, target_names=target_names))

    cm1 = confusion_matrix(y_pred_validation, validation_labels)
    
    print('confusion_matrix : \n', cm1)

    total1=sum(sum(cm1))

    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print('Sensitivity : ', sensitivity )

    Specificity = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    print('Specificity : ', Specificity )


save_bottleneck_features()
train_top_model()

print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')



/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/keras/applications/mobilenet.py:224: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (Activation)      (None, None, None, 32)    0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
__________

Found 400 images belonging to 2 classes.
 train_labels shape (1000,)
 train_labels : 
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

Epoch 39/50
1000/1000 [==============================] - 17s 17ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 40/50
1000/1000 [==============================] - 16s 16ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 41/50
1000/1000 [==============================] - 16s 16ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 42/50
1000/1000 [==============================] - 17s 17ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 43/50
1000/1000 [==============================] - 17s 17ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 44/50
1000/1000 [==============================] - 17s 17ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 45/50
1000/1000 [==============================] - 17s 17ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 46/50
1000/1000 [==================

/home/fatma-sayed/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


 y_pred_validation shape (400,)
 y_pred_validation : 
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0